In [19]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import * 
import cvzone



model=YOLO('yolov8s.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        print("hello")
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

# cap=cv2.VideoCapture('traffic.mp4')
cap=cv2.VideoCapture('traffic.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count=0

# Area of the particular region as you wish to detect 
# area=[(270,238),(285,262),(592,226),(552,207)]  
# area=[(200,238),(285,262),(592,226),(552,207)] 
area=[(200,313),(120,424),(950,424),(880,344)] 

# Tracking the particular object
tracker=Tracker()


area_c=set()

while True:

    # reading the vedio 
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
     #Skiping the unwanted frame 
    if count % 3 != 0:
        continue

    #resize the vedio frame as per our convience 
    frame=cv2.resize(frame,(1020,500))
    #cv2.polylines(frame,[np.array(area,np.int32)],True,(0,0,255),3)
    

    results=model.predict(frame)#show=True
    #print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
    print(px)
    list=[]
    

    for index,row in px.iterrows():
        # coordinate of the object (x1=xmin,y1=ymin,x2=xmax,y2=ymax)
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])

        # passing the index reference of class 
        d=int(row[5])
        c=class_list[d]

        # detecting the particular object like car
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    cv2.polylines(frame,[np.array(area,np.int32)],True,(255,255,0),3)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        center_x=int(x3+x4)//2
        center_y=int(y3+y4)//2
        results=cv2.pointPolygonTest(np.array(area,np.int32),((center_x,center_y)),False)
        if results >=0:
            cv2.circle(frame,(center_x,center_y),3,(0,0,255),-1)
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.polylines(frame,[np.array(area,np.int32)],True,(0,255,0),3)
            
            #cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
            area_c.add(id)
           
    #cv2.polylines(frame,[np.array(area,np.int32)],True,(255,255,0),3)
    car_count=(len(area_c))
    cvzone.putTextRect(frame,f'car_count:-{car_count}',(19,80),2,2)
    # cv2.putText(frame,str(count),(61,146),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),2)
    cv2.imshow("Vehicle Counter", frame)
    if cv2.waitKey(1)&0xFF==27:
        break

cap.release()
cv2.destroyAllWindows()

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']



0: 320x640 4 cars, 558.6ms
Speed: 7.8ms preprocess, 558.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



            0          1           2          3         4    5
0  538.966431  54.317383  565.311523  75.601692  0.733106  2.0
1  576.577881  51.809483  605.778992  73.071838  0.704393  2.0
2  525.903748  29.943554  548.069702  44.465607  0.624229  2.0
3  932.978638  29.421444  961.949402  42.913742  0.340827  2.0


0: 320x640 3 cars, 552.9ms
Speed: 1.8ms preprocess, 552.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



            0          1           2          3         4    5
0  539.159668  54.486965  566.810608  78.230118  0.734891  2.0
1  576.722595  53.762238  609.070557  75.240997  0.724758  2.0
2  527.157593  30.094074  549.170898  44.370304  0.667903  2.0


...

            0          1           2          3         4    5
0  485.707642  24.122868  507.288635  40.439667  0.531054  2.0
1  536.242188  27.351309  554.877808  43.283070  0.496932  2.0
2  552.997253  17.273254  569.538269  30.936504  0.432068  2.0
3  861.238953  29.294882  898.359436  47.851421  0.353266  2.0
4  773.455688   2.759498  799.947083  15.607700  0.258915  2.0


0: 320x640 6 cars, 453.7ms
Speed: 7.5ms preprocess, 453.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



            0          1           2          3         4    5
0  554.582886  19.559107  572.299133  32.528507  0.686202  2.0
1  536.075012  29.059679  555.212402  45.118000  0.653011  2.0
2  484.682861  25.144672  506.704132  41.766762  0.620388  2.0
3  850.592651  28.401073  884.061401  45.362537  0.471765  2.0
4  501.512177   9.372713  519.251099  27.344736  0.307114  2.0
5  768.601379   2.206012  791.670471  15.537226  0.287298  2.0


0: 320x640 3 cars, 477.2ms
Speed: 2.5ms preprocess, 477.2ms inference, 15.6ms postprocess per image at shape (1, 3, 320, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


            0          1           2          3         4    5
0  536.515503  30.894871  555.329773  47.902740  0.677129  2.0
1  555.764038  20.933477  574.222778  33.846905  0.633196  2.0
2  484.205292  27.673372  504.830841  44.228462  0.560901  2.0
